In [15]:
import os
import pandas as pd
from src.logger import logging
import pandas as pd
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from src.components.data_transformaion import DataTransformation
from scipy import sparse

from src.logger import logging
import pandas as pd
import numpy as np
from dataclasses import dataclass

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from catboost import CatBoostRegressor
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor


In [16]:
df = pd.read_csv('../data/train.csv')

In [17]:
train_set, test_set = train_test_split(df,test_size=0.2,random_state=42)

In [31]:
num_columns = ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
'MiscVal', 'MoSold', 'YrSold']
cat_columns = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
'SaleType', 'SaleCondition']

num_pipeline = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="median")),
        ('scaler', StandardScaler()),
    ]
)
cat_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ('scaler', StandardScaler(with_mean=False)),
    ]
)

preprocessor = ColumnTransformer(
    [
        ("num_pipepine", num_pipeline, num_columns),
        ("cat_pipeline", cat_pipeline, cat_columns),
])


In [32]:
target_column = 'SalePrice'

input_feature_train_df=train_set.drop(columns=[target_column],axis=1)
target_feature_train_df=train_set[target_column]

input_feature_test_df=test_set.drop(columns=[target_column],axis=1)
target_feature_test_df=test_set[target_column]

logging.info(
    f"Applying preprocessing object on training dataframe and testing dataframe."
)

input_feature_train_arr=preprocessor.fit_transform(input_feature_train_df)
input_feature_test_arr=preprocessor.transform(input_feature_test_df)



print(input_feature_train_arr.shape, np.array(target_feature_train_df).shape ,"-", input_feature_test_arr.shape, np.array(target_feature_test_df).shape)
print(type(input_feature_train_arr),type(input_feature_test_arr))


(1168, 286) (1168,) - (292, 286) (292,)
<class 'scipy.sparse._csr.csr_matrix'> <class 'scipy.sparse._csr.csr_matrix'>


In [34]:
# train_arr = np.c_[
#                 input_feature_train_arr, np.array(target_feature_train_df).astype(float)
#             ]
# input_feature_train_arr
a = input_feature_train_arr.toarray()
b = np.array(target_feature_train_df)
print(type(a),a.shape,type(b),b.shape)
# b = sparse.hstack(b)
# print(type(a),a.shape,type(b),b.shape)
# np.concatenate((a,np.concatenate(b)[:,None]),axis=1)
# np.concatenate((a,b[:,None]),axis=1)
# np.column_stack((a,b))
train_arr = np.c_[
                a,b
            ]
# train_arr
train_array = train_arr
type(train_array)

# a

<class 'numpy.ndarray'> (1168, 286) <class 'numpy.ndarray'> (1168,)


numpy.ndarray

In [35]:
test_arr = np.c_[input_feature_test_arr.toarray(), np.array(target_feature_test_df)]
test_array = test_arr
test_array

array([[ 3.81232378e-01, -8.66764305e-01, -1.24683632e-02, ...,
         2.63383911e+00,  0.00000000e+00,  1.54500000e+05],
       [ 8.82188480e-01,  7.41099591e-02,  1.23452047e+00, ...,
         2.63383911e+00,  0.00000000e+00,  3.25000000e+05],
       [-7.45330875e-01, -6.31545739e-01, -6.35962781e-01, ...,
         2.63383911e+00,  0.00000000e+00,  1.15000000e+05],
       ...,
       [ 1.69560786e-01,  7.41099591e-02, -3.24215572e-01, ...,
         2.63383911e+00,  0.00000000e+00,  1.89000000e+05],
       [-1.85577108e-01,  3.09328525e-01, -4.57821519e-01, ...,
         2.63383911e+00,  0.00000000e+00,  1.08000000e+05],
       [-1.85917410e-02, -8.66764305e-01, -1.24683632e-02, ...,
         2.63383911e+00,  0.00000000e+00,  1.24500000e+05]])

In [36]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    try:
        report = {}
        for i in range(len(models)):
            model = list(models.values())[i]
            # para=param[list(models.keys())[i]]

            # gs = GridSearchCV(model,para)
            # gs.fit(X_train,y_train)
            # model.set_params(**gs.best_params_)
            model.fit(X_train,y_train)

            y_train_pred = model.predict(X_train)

            y_test_pred = model.predict(X_test)

            train_model_score = r2_score(y_train, y_train_pred)

            test_model_score = r2_score(y_test, y_test_pred)

            report[list(models.keys())[i]] = test_model_score

        return report
    except Exception as e:
         raise e

In [38]:
try:
    X_train,y_train,X_test,y_test=(
    train_array[:,:-1],
    train_array[:,-1],
    test_array[:,:-1],
    test_array[:,-1]
    )

    models = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Linear Regression": LinearRegression(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
    }

    params={
    "Decision Tree": {
        'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        # 'splitter':['best','random'],
        # 'max_features':['sqrt','log2'],
    },
    "Random Forest":{
        # 'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        
        # 'max_features':['sqrt','log2',None],
        'n_estimators': [8,16,32,64,128,256]
    },
    "Gradient Boosting":{
        # 'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
        'learning_rate':[.1,.01,.05,.001],
        'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
        # 'criterion':['squared_error', 'friedman_mse'],
        # 'max_features':['auto','sqrt','log2'],
        'n_estimators': [8,16,32,64,128,256]
    },
    "Linear Regression":{},
    "XGBRegressor":{
        'learning_rate':[.1,.01,.05,.001],
        'n_estimators': [8,16,32,64,128,256]
    },
    "CatBoosting Regressor":{
        'depth': [6,8,10],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [30, 50, 100]
    },
    "AdaBoost Regressor":{
        'learning_rate':[.1,.01,0.5,.001],
        # 'loss':['linear','square','exponential'],
        'n_estimators': [8,16,32,64,128,256]
    }

    }

    model_report:dict=evaluate_model(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,
                                    models=models)#,param=params

    best_model_score = max(sorted(model_report.values()))

    ## To get best model name from dict

    best_model_name = list(model_report.keys())[
    list(model_report.values()).index(best_model_score)
    ]
    best_model = models[best_model_name]   


    if best_model_score<0.6:
        print('Best model not found')


    predicted=best_model.predict(X_test)

    r2_square = r2_score(y_test, predicted)
    print(r2_square, best_model)

except Exception as e:
    raise e
    

0.9063522938950872 GradientBoostingRegressor()
